# Demo code to import to a new project from a zip file

Process
1. If prj exists, delete prj
2. create a new prj, then import "export prj name"+_whl_prj.zip to new prj

In [1]:
import sys
import os
import yaml
import subprocess

In [2]:
# point to a local file with credentials. It is not synced to git.
CONFIG_FILE = "cpd484_jb_import.yaml"

In [3]:
# Load parameters from the YAML file
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

DSJOB_URL = config['url']
DSJOB_USER = config['user']
DSJOB_PWD = config['password']
EXPORT_PRJ_NAME = config['export_prj_name']
EXPORT_ZIP_NAME = EXPORT_PRJ_NAME+".zip"
IMPORT_PRJ_NAME = config['import_prj_name']

In [4]:
# Enable dsjob
%env CPDCTL_ENABLE_DSJOB=true
%env CPDCTL_ENABLE_DATASTAGE=true
%env CPDCTL_ENABLE_VOLUMES=1
!echo $CPDCTL_ENABLE_DSJOB
!echo $CPDCTL_ENABLE_DATASTAGE
!echo $CPDCTL_ENABLE_VOLUMES

env: CPDCTL_ENABLE_DSJOB=true
env: CPDCTL_ENABLE_DATASTAGE=true
env: CPDCTL_ENABLE_VOLUMES=1
true
true
1


In [5]:
# Configure cpdctl with the parameters
!cpdctl config user set CP4D-user --username "$DSJOB_USER" --password "$DSJOB_PWD"
!cpdctl config profile set CP4D-profile --url "$DSJOB_URL" --user CP4D-user
!cpdctl config profile use CP4D-profile

Switched to profile "CP4D-profile".


In [6]:
# list all projects
!cpdctl dsjob list-projects

...
Business Catalog Enrichment
GDPR Lineage
test_python_pipeline
DQ Definition_Rules
Multicloud Data Integration L3 Tech Lab
Data Replication Lab
Auto Policy Risk
evaluate-an-ml-model
Data-Science-and-MLOps
DataStage Import

Total: 10 Projects

Status code = 0


In [7]:
# check if import prj exists or not
#!cpdctl project list --name "$IMPORT_PRJ_NAME" --match exact

import subprocess

# Execute the command
command = f'cpdctl project list --name "{IMPORT_PRJ_NAME}" --match exact'
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# Capture the return code and output
return_code = result.returncode
output = result.stdout

# Print the return code and output
print("Return Code:", return_code)
print("Output:", output)

# Check if the output contains "Nothing to show"
if "Nothing to show" in output:
    print("The project does not exist or there was an error.")

else:
    print("The project exists.")
    # if exist delete project
    !cpdctl dsjob delete-project --project "$IMPORT_PRJ_NAME"


Return Code: 0
Output: ...
ID                                     Name               Created                    Description                                  Type
fd46b0f1-8489-4393-bdae-559bb24ea53f   DataStage Import   2024-05-30T06:49:47.360Z   This Project is created using dsjob plugin   cpd

The project exists.
...
{
    "StatusCode": 204,
    "Headers": {
        "Date": [
            "Thu, 30 May 2024 06:59:44 GMT"
        ],
        "Server": [
            "---"
        ],
        "Set-Cookie": [
            "2ac1df5a53d05af1ed3c0e46006c8757=45ff8c8165de5523041d91b9730046da; path=/; HttpOnly; Secure; SameSite=Lax"
        ],
        "Strict-Transport-Security": [
            "max-age=31536000; includeSubDomains",
            "max-age=31536000; includeSubDomains"
        ],
        "X-Frame-Options": [
            "SAMEORIGIN"
        ]
    },
    "Result": null,
    "RawResult": null
}



In [11]:
# If you don't mind try 5 times to delete a non-existing project, below will do as well.
# !cpdctl dsjob delete-project --project $IMPORT_PRJ_NAME

...
20240530171316  retry  0  err  expected only one project but got 0
20240530171318  retry  1  err  expected only one project but got 0
20240530171322  retry  2  err  expected only one project but got 0
20240530171330  retry  3  err  expected only one project but got 0
20240530171347  retry  4  err  expected only one project but got 0
FAILED
Error running dsjob: Finding Project:
Failed to find the Project



In [8]:
# Create prj
!cpdctl dsjob create-project --name "$IMPORT_PRJ_NAME" 

126e8e2d-c306-41e4-855a-c0497c69bd88


In [9]:
!cpdctl dsjob import --project "$IMPORT_PRJ_NAME" --import-file "$EXPORT_ZIP_NAME" --wait -1

...

ID: 40705b6c-d043-456f-b8e6-ea617cb2a2f4
running
completed

Status code =  0
